#DATASCI W261: Machine Learning at Scale

#This notebook shows a Hadoop MapReduce job of WordCount.

##Data

In [1]:
%%writefile wordcount.txt
hello hi hi hallo
bonjour hola hi ciao
nihao konnichiwa ola
hola nihao hello

Overwriting wordcount.txt


##Mapper

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


##Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


##Test Code

In [4]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort -k1,1 | python reducer.py

bar	1
foo	3
labs	1
quux	2


## Run wordcount in hadoop

###start yarn and hdfs

In [2]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/yarn-liang-resourcemanager-ldai.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/yarn-liang-nodemanager-ldai.out
15/08/21 06:23:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-namenode-ldai.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-datanode-ldai.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-secondarynamenode-ldai.out
15/08/21 06:23:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###create a folder 

In [3]:
#!hdfs dfs -mkdir -p /user/liang

15/08/21 06:23:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###upload wordcount.txt to hdfs

In [13]:
!hdfs dfs -put wordcount.txt 


16/02/22 19:21:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `wordcount.txt': File exists
16/02/22 19:21:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted wordcountOutput


### Hadoop streaming command

In [5]:
!hdfs dfs -rm -r wordcountOutput

!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input wordcount.txt \
-output wordcountOutput

16/02/26 15:32:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted wordcountOutput
16/02/26 15:32:34 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/26 15:32:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar9149739812416871382/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob4624454949110693741.jar tmpDir=null


###show the results

In [6]:
!hdfs dfs -cat wordcountOutput/part-00000

16/02/26 15:33:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


### Use a directory as input

In [8]:
!hdfs dfs -rm -r wordcountOutput
!hdfs dfs -rm -r wordcountInputDir
!hdfs dfs -mkdir wordcountInputDir
!hdfs dfs -put wordcount.txt wordcountInputDir/wordcount1.txt
!hdfs dfs -put wordcount.txt wordcountInputDir/wordcount2.txt

!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input wordcountInputDir \
-output wordcountOutput

16/02/26 15:36:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `wordcountOutput': No such file or directory
16/02/26 15:36:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `wordcountInputDir': No such file or directory
16/02/26 15:36:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:36:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:36:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:36:38 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/26 15:36:38 WARN util.NativeCodeLoader: 

In [9]:
!hdfs dfs -cat wordcountOutput/part-00000

16/02/26 15:37:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bonjour	2
ciao	2
hallo	2
hello	4
hi	6
hola	4
konnichiwa	2
nihao	4
ola	2


### Hadoop streaming can also handle zipped files in a folder

In [14]:
!hdfs dfs -rm -r wordcountOutput
!hdfs dfs -rm -r wordcountInputZipDir
!hdfs dfs -mkdir wordcountInputZipDir
!cp -f wordcount.txt wordcount2.txt
!gzip -f -k wordcount.txt wordcount2.txt
!hdfs dfs -put wordcount.txt.gz wordcountInputZipDir/
!hdfs dfs -put wordcount2.txt.gz wordcountInputZipDir/

!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input wordcountInputZipDir \
-output wordcountOutput

16/02/26 15:43:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `wordcountOutput': No such file or directory
16/02/26 15:43:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted wordcountInputZipDir
16/02/26 15:43:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:43:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:43:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/26 15:43:10 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/26 15:43:10 WARN util.NativeCodeLoader: Unable to load native-

In [15]:
!hdfs dfs -cat wordcountOutput/part-00000

16/02/26 15:43:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bonjour	2
ciao	2
hallo	2
hello	4
hi	6
hola	4
konnichiwa	2
nihao	4
ola	2


###stop yarn and hdfs

In [11]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop
16/01/20 19:08:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/20 19:09:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
